<a href="https://colab.research.google.com/github/datarobot-community/custom-models/blob/master/custom_inference/python/boston_housing/Main_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DRUM - Automated Model Serving Made Easy

 We'll get our hands dirty by 

* Building a simple regression model using Scikit
* Using DRUM for Batch Scoring
* Using DRUM to get a REST API endpoint
* Showing a simple example app connected to the REST API
* Reviewing support for various model frameworks (e.g., H2O, Keras, XGBoost, and DataRobot)
* Monitoring with the MLOps agent

## Build a Model

In [1]:
!git clone https://github.com/datarobot-community/mlops-examples.git

Cloning into 'mlops-examples'...
remote: Enumerating objects: 245, done.
remote: Counting objects: 100% (245/245), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 245 (delta 96), reused 229 (delta 80), pack-reused 0
Receiving objects: 100% (245/245), 18.57 MiB | 40.45 MiB/s, done.
Resolving deltas: 100% (96/96), done.


In [2]:
!pip install -r /content/mlops-examples/custom_inference/python/boston_housing/colab-requirements.txt -q

     |████████████████████████████████| 276kB 8.8MB/s 
     |████████████████████████████████| 8.7MB 8.9MB/s 
     |████████████████████████████████| 276kB 54.7MB/s 
     |████████████████████████████████| 148.9MB 53kB/s 
     |████████████████████████████████| 61kB 10.0MB/s 
     |████████████████████████████████| 204kB 57.3MB/s 
     |████████████████████████████████| 788kB 51.4MB/s 
     |████████████████████████████████| 153kB 55.5MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 808kB 51.4MB/s 
     |████████████████████████████████| 204kB 52.4MB/s 
     |████████████████████████████████| 112kB 57.2MB/s 
     |████████████████████████████████| 552kB 58.8MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import pickle
import datetime

## load data

df = pd.read_csv(
    '/content/mlops-examples/custom_inference/python/boston_housing/data/boston_housing.csv'
    )
df.head()

## set features and target

X = df.drop('MEDV', axis=1)
y = df['MEDV']

## train the model
rf = RandomForestRegressor(n_estimators = 20)
rf.fit(X,y)

## serialize the model

with open('/content/mlops-examples/custom_inference/python/boston_housing/src/custom_model/rf.pkl', 'wb') as pkl:
    pickle.dump(rf, pkl)

# Testing

You can test how the model performs and get its latency times and memory usage.
In this mode, the model is started with a prediction server. Different request combinations are submitted to it. After it completes, it returns a report.

In [11]:
%%sh 
cd "/content/mlops-examples/custom_inference/python/boston_housing" && 
drum perf-test --code-dir ./src/custom_model --input ./data/boston_housing_inference.csv --target-type regression

Preparing test data...



Running test case: 72 bytes - 1 samples, 100 iterations
Running test case: 0.1MB - 1449 samples, 50 iterations
Running test case: 10MB - 144964 samples, 5 iterations
Running test case: 50MB - 724823 samples, 1 iterations

  size     samples   iters    min     avg     max    used (MB)   total (MB)
72 bytes         1     100   0.009   0.009   0.019     489.445    13021.090
0.1MB         1449      50   0.015   0.017   0.022     494.137    13021.090
10MB        144964       5   0.658   0.670   0.681     556.539    13021.090
50MB        724823       1   3.332   3.332   3.332     745.023    13021.090


2020-11-21 19:07:41.491115: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
tput: terminal attributes: No such device or address



# Validation

You can validate the model on a set of various checks. It is highly recommended to run these checks, as they are performed in DataRobot before the model can be deployed.

List of checks:

* null values imputation: each feature of the provided dataset is set to missing and fed to the model.

In [14]:
%%sh 
cd "/content/mlops-examples/custom_inference/python/boston_housing" && 
drum validation --code-dir ./src/custom_model --input ./data/boston_housing_inference.csv --target-type regression > validation.log

2020-11-21 19:12:04.192234: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
2020-11-21 19:12:07.816878: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
2020-11-21 19:12:11.435649: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/distributed/confi

In [21]:
%%sh 
cd "/content/mlops-examples/custom_inference/python/boston_housing" && 
tail -15 validation.log

0       26.850
1       26.080
2       35.945
3       33.580
4       35.960
5       29.760
6       23.830
7       24.745
8       24.825


Validation checks results
      Test case         Status
Null value imputation   PASSED


# Batch Scoring with DRUM
<a id="setup_complete"></a>

At this point our model has been written to disk and we want to use it to make predictions.  To do this, we'll leverage DRUM and its ability to natively handle our Scikit-Learn model. All we need to do is tell DRUM where the model resides and what data we wish to score.  

DRUM provides native support for many frameworks. To use DRUM with model frameworks that are not supported out-of-the box, we'll just need to create some custom hooks so DRUM.  In this example, we'll explain some very simple custom hooks and provide links to more complex examples.  

In [22]:
!drum score --code-dir /content/mlops-examples/custom_inference/python/boston_housing/src/custom_model --input /content/mlops-examples/custom_inference/python/boston_housing/data/boston_housing_inference.csv --output /content/mlops-examples/'Custom Model Examples'/'Boston Housing'/data/predictions.csv --target-type regression

/usr/local/lib/python3.6/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
2020-11-21 19:17:57.300319: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [23]:
pd.read_csv("/content/mlops-examples/custom_inference/python/boston_housing/data/predictions.csv").head()

,Predictions
0,25.610
1,22.370
2,35.155
3,33.580
4,35.780


# Start the inference server locally

Batch scoring is very useful; however, the value of DRUM does not stop there.  We can also leverage DRUM to serve our model as a RESTful API endpoint.  The only thing that changes is the way we will structure the command: using the `server` mode instead of `score` mode.  We'll also need to provide an address which is NOT in use.  

When starting the server, we'll use `subprocess.Popen` so we may interact with the server in this notebook.

In [24]:
import subprocess
import requests
import pandas as pd
from io import BytesIO
import yaml
import time
import os
import datarobot as dr
from pprint import pprint

In [25]:
run_inference_server = ["drum",
              "server",
              "--code-dir","/content/mlops-examples/custom_inference/python/boston_housing/src/custom_model", 
              "--address", "0.0.0.0:6789", 
              "--show-perf",
              "--target-type", "regression",
              "--logging-level", "info",
              "--show-stacktrace",
              "--verbose"
              ]

In [26]:
inference_server = subprocess.Popen(run_inference_server, stdout=subprocess.PIPE)

## Ping the Server to make sure it is running

In [27]:
## confirm the server is running
time.sleep(5) ## snoozing before pinging the server to give it time to actually start
print('check status')
requests.request("GET", "http://0.0.0.0:6789/").content

check status


b'{"message":"OK"}\n'

## Send data to server for inference

The request must provide our dataset as form data.  In order to do so, we'll create a simple Python function to pass the data over appropriately.

In [28]:
def score(data, port = "6789"):
    b_buf = BytesIO()
    b_buf.write(data.to_csv(index=False).encode("utf-8"))
    b_buf.seek(0)
  
    url = "http://localhost:{}/predict/".format(port)
    files = [
        ('X', b_buf)
    ]
    response = requests.request("POST", url, files = files, timeout=None, verify=False)
    return response

In [29]:
# %%timeit
scoring_data = pd.read_csv("/content/mlops-examples/custom_inference/python/boston_housing/data/boston_housing_inference.csv")
predictions = score(scoring_data).json() ## score entire dataset but only show first 5 records
pprint(predictions)

{'predictions': [25.61,
                 22.37,
                 35.155,
                 33.58,
                 35.78,
                 27.92,
                 21.51,
                 24.265,
                 16.445]}


In [30]:
requests.request("GET", "http://0.0.0.0:6789/").content

b'{"message":"OK"}\n'

In [31]:
requests.request("POST", "http://0.0.0.0:6789/shutdown/").content

b'Server shutting down...'

## Value Prop

Still wondering why DRUM is beneficial?  First, you don't need to write an API to get a model up and running. Second, DRUM allows you to abstract away the framework (provided you're using one that is natively supported, or that you can write enough Python to instruct DRUM to hook up to the model).  

For example, you can hot-swap models as needed (see examples in `./src/other_models`). 

Below we run through several other frameworks within `score` -- these are also supported in `server` mode!

#### H2O Mojo

In [32]:
!drum score --code-dir /content/mlops-examples/custom_inference/python/boston_housing/src/other_models/h2o_mojo/regression --input /content/mlops-examples/custom_inference/python/boston_housing/data/boston_housing_inference.csv --target-type regression


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
   Predictions
0    24.504000
1    22.492000
2    34.554001
3    34.420001
4    35.289001
5    28.394001
6    21.936000
7    23.451000
8    17.065000


#### Keras

In [33]:
!drum score --code-dir /content/mlops-examples/custom_inference/python/boston_housing/src/other_models/python3_keras_joblib --input /content/mlops-examples/custom_inference/python/boston_housing/data/boston_housing_inference.csv --target-type regression


2020-11-21 19:18:43.315510: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SimpleImputer from version 0.23.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.23.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying t

#### XGBoost

Requires XGBoost

In [34]:
!drum score --code-dir /content/mlops-examples/custom_inference/python/boston_housing/src/other_models/python3_xgboost --input /content/mlops-examples/custom_inference/python/boston_housing/data/boston_housing_inference.csv --target-type regression


/usr/local/lib/python3.6/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
2020-11-21 19:18:54.373380: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
   Predictions
0    24.541843
1    21.260277
2    34.018497
3    32.569200
4    34.248066
5    27.282364
6    20.803959
7    19.645220
8    16.968880


#### DataRobot Codegen

In [35]:
!drum score --code-dir /content/mlops-examples/custom_inference/python/boston_housing/src/other_models/dr_codegen --input /content/mlops-examples/custom_inference/python/boston_housing/data/boston_housing_inference.csv --target-type regression


   Predictions
0    24.258228
1    24.258228
2    32.451515
3    32.451515
4    32.451515
5    24.258228
6    21.078378
7    13.107812
8    13.107812


# Monitoring Deployments

What follows will require a DataRobot account.  You can get a trial account at [https://www.datarobot.com/trial/](https://www.datarobot.com/trial/). 

Also, JDK 11 or 12 will be required.

In this example, we start an agent service locally to monitor a spooler.  The spooler could be something as simple as a local file system, or a little more realistic like a message broker (pubsub, rabbitmq, sqs).  

Once the agent is spun-up locally, we enable a few environment variables to let DRUM know that there is an agent present and that it needs to buffer data to the defined spool.  

## Getting the MLOps agent
We grab the agent through the [DataRobot UI](https://app2.datarobot.com/account/developer-tools). (This link is for the DatRobot Trial. If you have a Managed Cloud or On-prem license, use that to navigate to Developer Tools and select to download External Monitoring Agent.)



In [ ]:
token = "YOUR_DATAROBOT_API_KEY"
endpoint = "https://app2.datarobot.com"
## connect to DataRobot platform with python client. 
client = dr.Client(token, "{}/api/v2".format(endpoint))
# mlops_agents_tb = client.get("mlopsInstaller")
# with open("/content/odsc-ml-drum/mlops-agent.tar.gz", "wb") as f:
#     f.write(mlops_agents_tb.content)

In [ ]:
!tar -xf /content/datarobot-mlops-agent-6.2.4-399.tar.gz -C .

## Configuring the Agent

To configure the agent, we just need to define the DataRobot MLOps location and our API token.  By default, the agent expects the data to be spooled on the local file system.  Make sure that default location (`/tmp/ta`) exists.

In [ ]:
!mkdir -p /tmp/ta

In [ ]:
agents_dir = "/content/datarobot-mlops-agent-6.2.4"
with open(r'{}/conf/mlops.agent.conf.yaml'.format(agents_dir)) as file:
    documents = yaml.load(file, Loader=yaml.FullLoader)
## Configure the location of the mlops instance with which we'll communicate
documents['mlopsUrl'] = endpoint
# Set your API token
documents['apiToken'] = token
## Write the configuration back to disk
with open('../{}/conf/mlops.agent.conf.yaml'.format(agents_dir), "w") as f:
    yaml.dump(documents, f)

## Start the Agent Service

Here we're checking to make sure we can start up the agent's service.  

This will require JDK 11 or JDK 12 (these are the tested versions).

In [ ]:
## run agents service
subprocess.call("{}/bin/start-agent.sh".format(agents_dir))

In [ ]:
## check status
check = subprocess.Popen(["../{}/bin/status-agent.sh".format(agents_dir)], stdout=subprocess.PIPE)
print(check.stdout.readlines())
check.terminate()

In [ ]:
## check log to see that the agent connected to DR MLOps
check = subprocess.Popen(["cat", "../{}/logs/mlops.agent.log".format(agents_dir)], stdout=subprocess.PIPE)
for line in check.stdout.readlines():
    print(line)
check.terminate()

## DataRobot MLOps - Deploying External Model 
To communicate with DataRobot MLOps, we need to install the MLOps Python client provided in the downloaded tarball.

In [ ]:
!pip install /content/datarobot_mlops_package-*/lib/datarobot*.whl -q

In [ ]:
from datarobot.mlops.mlops import MLOps
from datarobot.mlops.common.enums import OutputType
from datarobot.mlops.connected.client import MLOpsClient
from datarobot.mlops.common.exception import DRConnectedException
from datarobot.mlops.constants import Constants

In [ ]:
DEPLOYMENT_NAME="Boston Housing Prices PGH Data Science Meetup"
TRAINING_DATA = '/content/mlops-examples/custom_inference/python/boston_housing/data/boston_housing.csv''

In [ ]:
model_info = {
        "name": "Boston Housing Pricing",
        "modelDescription": {
            "description": "prediction price of home"
        },
        "target": {
            "type": "Regression",
            "name": "MEDV",
        }
}

In [ ]:
# Create and connect the client
mlops_client = MLOpsClient(endpoint, token)

# Add training_data to model configuration
print("Uploading training data - {}. This may take some time...".format(TRAINING_DATA))
dataset_id = mlops_client.upload_dataset(TRAINING_DATA)
print("Training dataset uploaded. Catalog ID {}.".format(dataset_id))
model_info["datasets"] = {"trainingDataCatalogId": dataset_id}

# Create the model package
print('Create model package')
model_pkg_id = mlops_client.create_model_package(model_info)
model_pkg = mlops_client.get_model_package(model_pkg_id)
model_id = model_pkg["modelId"]

# Deploy the model package
print('Deploy model package')
deployment_id = mlops_client.deploy_model_package(model_pkg["id"],
                                                            DEPLOYMENT_NAME)

# Enable data drift tracking
print('Enable feature drift')
enable_feature_drift = TRAINING_DATA is not None
mlops_client.update_deployment_settings(deployment_id, target_drift=True,
                                                  feature_drift=enable_feature_drift)
_ = mlops_client.get_deployment_settings(deployment_id)

print("\nDone.")
print("DEPLOYMENT_ID=%s, MODEL_ID=%s" % (deployment_id, model_id))

DEPLOYMENT_ID = deployment_id
MODEL_ID = model_id

In [ ]:
from IPython.core.display import display, HTML
link = "{}/deployments/{}/overview".format(endpoint,deployment_id)
# display(HTML("""<a href="{link}">{link}</a>""".format( link=link )))
print(link)

# Adding Monitoring with MLOps Agent

## Monitoring with DRUM

There are a few additional parameters that we should set for the command line utility; or we can just create environment variables and allow DRUM to pick up the details from there.  

```
  --monitor             Monitor predictions using DataRobot MLOps. True or
                        False. (env: MONITOR). Monitoring cannot be used in
                        unstructured mode.
  --deployment-id DEPLOYMENT_ID
                        Deployment ID to use for monitoring model predictions
                        (env: DEPLOYMENT_ID)
  --model-id MODEL_ID   MLOps model ID to use for monitoring predictions (env:
                        MODEL_ID)
  --monitor-settings MONITOR_SETTINGS
                        MLOps setting to use for connecting with the MLOps
                        agent (env: MONITOR_SETTINGS)
```
For this example, we'll just set environment variables to add monitoring. 


In [ ]:
os.environ["MONITOR"] = "True"
os.environ["DEPLOYMENT_ID"] = deployment_id
os.environ["MODEL_ID"] = model_id
os.environ["MONITOR_SETTINGS"] = "spooler_type=filesystem;directory=/tmp/ta;max_files=5;file_max_size=1045876000"

In [ ]:
run_inference_server = ["drum",
              "server",
              "--code-dir","/content/mlops-examples/custom_inference/python/boston_housing/src/custom_model", 
              "--address", "0.0.0.0:43210", 
              "--show-perf",
              "--target-type", "regression",
              "--logging-level", "info",
              "--show-stacktrace",
#               "--verbose"
              ]

In [ ]:
inference_server_with_monitoring = subprocess.Popen(run_inference_server, stdout=subprocess.PIPE)

In [ ]:
predictions = score(
    pd.read_csv("/content/mlops-examples/custom_inference/python/boston_housing/data/boston_housing.csv").drop(["MEDV"],axis=1).head(100),
    "43210")

In [ ]:
pd.DataFrame(predictions.json()).head()

In [ ]:
requests.post("http://localhost:43210/shutdown/").content

In [ ]:
subprocess.call("../{}/bin/stop-agent.sh".format(agents_dir))

In [ ]:
## check that agent is stopped 
check = subprocess.Popen(["../{}/bin/status-agent.sh".format(agents_dir)], stdout=subprocess.PIPE)
print(check.stdout.readlines())
check.terminate()

In [ ]:
deployment = dr.Deployment.get(deployment_id)
deployment.get_service_stats()

In [ ]:
service_stats = deployment.get_service_stats()
service_stats.metrics